In [ ]:
import requests
import pandas as pd


API_KEY = "0EP6T2QSUFFJ7W2F"
symbols = ["AAPL", "MSFT"]  # Apple & Microsoft

numeric_cols = [
    "totalRevenue", "grossProfit", "operatingIncome", "netIncome",
    "researchAndDevelopment", "sellingGeneralAndAdministrative",
    "incomeTaxExpense", "ebit", "ebitda", "depreciationAndAmortization"
]

all_data = []

for symbol in symbols:
    print(f"Fetching data for {symbol} ...")
    url = f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={API_KEY}"
    response = requests.get(url)
    data = response.json()

    # Extract annual and quarterly reports
    annual = pd.DataFrame(data.get("annualReports", []))
    annual["type"] = "annual"
    annual["symbol"] = symbol

    quarterly = pd.DataFrame(data.get("quarterlyReports", []))
    quarterly["type"] = "quarterly"
    quarterly["symbol"] = symbol

    df = pd.concat([annual, quarterly], ignore_index=True)

    # Convert numeric columns
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # Convert date column
    if "fiscalDateEnding" in df.columns:
        df["fiscalDateEnding"] = pd.to_datetime(df["fiscalDateEnding"])

    all_data.append(df)

# Combine all company data
final_df = pd.concat(all_data, ignore_index=True)

# Save to CSV (in Colab environment)
final_df.to_csv("/content/financials_AAPL_MSFT.csv", index=False)

print("✅ Data extraction complete.")
final_df.head()


Fetching data for AAPL ...
Fetching data for MSFT ...
✅ Data extraction complete.


,fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,...,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,type,symbol
0,2024-09-30,USD,180683000000,391035000000,210352000000,210352000000,123216000000,26097000000,31370000000,57467000000,...,123485000000,29749000000,None,93736000000,None,123216000000,134661000000,93736000000,annual,AAPL
1,2023-09-30,USD,169148000000,383285000000,214137000000,214137000000,114301000000,24932000000,29915000000,54847000000,...,113736000000,16741000000,None,96995000000,None,114301000000,125820000000,96995000000,annual,AAPL
2,2022-09-30,USD,170782000000,394328000000,223546000000,223546000000,119437000000,25094000000,26251000000,51573000000,...,119103000000,19300000000,None,99803000000,None,119437000000,130541000000,99803000000,annual,AAPL
3,2021-09-30,USD,152836000000,365817000000,212981000000,212981000000,108949000000,21973000000,21914000000,43887000000,...,109207000000,14527000000,None,94680000000,None,111852000000,123136000000,94680000000,annual,AAPL
4,2020-09-30,USD,104956000000,274515000000,169559000000,169559000000,66288000000,19916000000,18752000000,38668000000,...,67091000000,9680000000,None,57411000000,None,69964000000,81020000000,57411000000,annual,AAPL


In [ ]:
from google.colab import files
files.download("/content/financials_AAPL_MSFT.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Import necessary libraries (if not already imported in previous cells)
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

# Use the final_df you created earlier
# Make a copy to avoid changing the original DataFrame
data_for_forecast = final_df.copy()

# 1. Filter for only ANNUAL reports
data_for_forecast = data_for_forecast[data_for_forecast['type'] == 'annual'].copy()

# 2. Extract Year from fiscalDateEnding
# Ensure the date column is in datetime format first
data_for_forecast['fiscalDateEnding'] = pd.to_datetime(data_for_forecast['fiscalDateEnding'])
data_for_forecast['Year'] = data_for_forecast['fiscalDateEnding'].dt.year

# 3. Filter for the years 2010 to 2024
data_for_forecast = data_for_forecast[(data_for_forecast['Year'] >= 2010) & (data_for_forecast['Year'] <= 2024)].copy()

# 4. Keep only essential columns and rename them
# Ensure 'totalRevenue' is numeric
data_for_forecast['totalRevenue'] = pd.to_numeric(data_for_forecast['totalRevenue'], errors='coerce')
# Convert to billions BEFORE forecasting
data_for_forecast['Revenue'] = data_for_forecast['totalRevenue'] / 1_000_000_000

data_for_forecast = data_for_forecast[['Year', 'symbol', 'Revenue']].copy()
data_for_forecast.rename(columns={'symbol': 'Company'}, inplace=True)

# Display the final data we'll use for forecasting
print("Cleaned data ready for forecasting:")
print(data_for_forecast.head())
print("\nLast few rows:")
print(data_for_forecast.tail())

Cleaned data ready for forecasting:
   Year Company  Revenue
0  2024    AAPL  391.035
1  2023    AAPL  383.285
2  2022    AAPL  394.328
3  2021    AAPL  365.817
4  2020    AAPL  274.515

Last few rows:
     Year Company  Revenue
112  2014    MSFT   86.833
113  2013    MSFT   77.849
114  2012    MSFT   73.723
115  2011    MSFT   69.943
116  2010    MSFT   62.484


In [ ]:
# Create a Linear Regression model instance
model = LinearRegression()

# Define the future years we want to predict
future_years = np.array([2025, 2026, 2027, 2028]).reshape(-1, 1) # Reshape for the model

all_forecasts = []

# Loop through each company (AAPL and MSFT)
for company in data_for_forecast['Company'].unique():
    print(f"\nForecasting for {company}...")

    # Get the historical data for the current company
    company_data = data_for_forecast[data_for_forecast['Company'] == company].sort_values('Year')

    # Prepare data for the model
    # Features: Year (needs to be a 2D array for scikit-learn)
    X_hist = company_data['Year'].values.reshape(-1, 1)
    # Target: Revenue
    y_hist = company_data['Revenue'].values

    # Train the Linear Regression model
    model.fit(X_hist, y_hist)

    # Predict revenue for the future years
    y_pred_future = model.predict(future_years)

    # Create DataFrames for historical and future data
    hist_df = company_data[['Year', 'Company', 'Revenue']].copy()
    hist_df['Type'] = 'Actual' # Label historical data

    future_df = pd.DataFrame({
        'Year': future_years.flatten(), # Flatten back to 1D array
        'Company': company,
        'Revenue': y_pred_future,
        'Type': 'Forecast' # Label forecast data
    })

    # Combine historical and forecast data for this company
    company_forecast = pd.concat([hist_df, future_df], ignore_index=True)
    all_forecasts.append(company_forecast)

# Combine the forecasts for both companies into one final DataFrame
final_forecast_df = pd.concat(all_forecasts, ignore_index=True)

# Display the results
print("\n--- Forecast Results ---")
# Show the last few actual years and the forecast years
print(final_forecast_df.tail(10)) # Shows last 2 actual + 4 forecast years * 2 companies

# Save the forecast results to a CSV file in Colab
forecast_file_path = '/content/python_revenue_forecast.csv'
final_forecast_df.to_csv(forecast_file_path, index=False)

print(f"\n✅ Forecast complete. Results saved to {forecast_file_path}")




Forecasting for AAPL...

Forecasting for MSFT...

--- Forecast Results ---
    Year Company     Revenue      Type
28  2019    MSFT  125.843000    Actual
29  2020    MSFT  143.015000    Actual
30  2021    MSFT  168.088000    Actual
31  2022    MSFT  198.270000    Actual
32  2023    MSFT  211.915000    Actual
33  2024    MSFT  245.122000    Actual
34  2025    MSFT  219.829057  Forecast
35  2026    MSFT  231.851439  Forecast
36  2027    MSFT  243.873821  Forecast
37  2028    MSFT  255.896204  Forecast

✅ Forecast complete. Results saved to /content/python_revenue_forecast.csv


In [ ]:
from google.colab import files
files.download('/content/python_revenue_forecast.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>